In [2]:
!pip3 install tensorflow_datasets

     |████████████████████████████████| 3.3 MB 942 kB/s eta 0:00:01     |████████████████████▊           | 2.1 MB 942 kB/s eta 0:00:02
     |████████████████████████████████| 829 kB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 3.6 MB/s  eta 0:00:01
     |████████████████████████████████| 177 kB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 100 kB 5.3 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=493275 sha256=6bb67ea31efe811ff32e2ffb724c733a58e00c9d16ac5c95d6bd81e6ab9fba47
  Stored in directory: /root/.cache/pip/wheels/6e/9c/ed/4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=23950 sha256=1e88cb6891ba9ed281d4c2e21cf4779b75466233378e1f0f6e65cb97c79c47ce
  Stored in directory: /root/.cache/pip/wheels/59/9a/1d/3f1afbbb5122d0410547bf9eb50955f4a7a98e53a6d8b99bd1
  Created wheel for dill: filename=dill-0.3.2-py3-none-any.whl size=81196 sha

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds
import tensorflow as tf

import time
import numpy as np
import matplotlib.pyplot as plt

* positional encoding

In [6]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

In [7]:
def scaled_dot_product_attention(query, key, value, mask): # Q 행렬, K 행렬, V 행렬, 마스크 수행 여부
  matmul_qk = tf.matmul(query, key, transpose_b=True)
  # Q 행렬과 K 행렬을 곱한다. 즉, 어텐션 스코어 행렬을 얻는다.

  dk = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(dk)
  # 스케일링.

  if mask is not None:
    logits += (mask * -1e9)
  # 필요하다면 마스크를 수행한다. 해당 조건문이 어떤 의미인지는 뒤에서 설명하며 현재는 무시.

  attention_weights = tf.nn.softmax(logits, axis=-1)
  # 소프트맥스 함수를 사용하여 어텐션 가중치들. 즉, 어텐션 분포를 얻는다.

  output = tf.matmul(attention_weights, value)
  # 어텐션 분포 행렬과 V 행렬을 곱하여 최종 결과를 얻는다.

  return output, attention_weights
  # 최종 결과와 어텐션 분포 리턴. 어텐션 분포 또한 리턴하는 이유는 아래에서 값을 출력해보며 함수 테스트를 위함.

* scaled_dot_product 함수 테스트

In [9]:
# 임의의 Query, Key, Value인 Q, K, V 행렬 생성
np.set_printoptions(suppress=True)
temp_k = tf.constant([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[   1,0],
                      [  10,0],
                      [ 100,5],
                      [1000,6]], dtype=tf.float32)  # (4, 2)
temp_q = tf.constant([[0, 10, 0]], dtype=tf.float32)  # (1, 3)

In [10]:
temp_out, temp_attn = scaled_dot_product_attention(temp_q, temp_k, temp_v, None)
print(temp_attn) # 어텐션 분포(어텐션 가중치의 나열)
print(temp_out) # 어텐션 값

tf.Tensor([[0. 1. 0. 0.]], shape=(1, 4), dtype=float32)
tf.Tensor([[10.  0.]], shape=(1, 2), dtype=float32)


In [11]:
temp_q = tf.constant([[0, 0, 10]], dtype=tf.float32)
temp_out, temp_attn = scaled_dot_product_attention(temp_q, temp_k, temp_v, None)
print(temp_attn) # 어텐션 분포(어텐션 가중치의 나열)
print(temp_out) # 어텐션 값

tf.Tensor([[0.  0.  0.5 0.5]], shape=(1, 4), dtype=float32)
tf.Tensor([[550.    5.5]], shape=(1, 2), dtype=float32)


In [12]:
temp_q = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=tf.float32)  # (3, 3)
temp_out, temp_attn = scaled_dot_product_attention(temp_q, temp_k, temp_v, None)
print(temp_attn) # 어텐션 분포(어텐션 가중치의 나열)
print(temp_out) # 어텐션 값

tf.Tensor(
[[0.  0.  0.5 0.5]
 [0.  1.  0.  0. ]
 [0.5 0.5 0.  0. ]], shape=(3, 4), dtype=float32)
tf.Tensor(
[[550.    5.5]
 [ 10.    0. ]
 [  5.5   0. ]], shape=(3, 2), dtype=float32)


* Multi head attention 전체 구현 -> 각 인코더 층의 첫번째 sublayer

In [15]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"): # 정의하기
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads # 8
    self.d_model = d_model # 512

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model) #WQ
    self.key_dense = tf.keras.layers.Dense(units=d_model) #WK
    self.value_dense = tf.keras.layers.Dense(units=d_model) #WV

    self.dense = tf.keras.layers.Dense(units=d_model) #WO

  def split_heads(self, inputs, batch_size): # 아래의 call 함수에서 헤드를 나누기 위해서 호출
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # 1. WQ, WK, WV에 해당하는 밀집층 지나기
    query = self.query_dense(query) # (batch_size, seq_len, d_model) 
    key = self.key_dense(key) # (batch_size, seq_len, d_model)
    value = self.value_dense(value) # (batch_size, seq_len, d_model)

    # 2. 헤드 나누기 (split_heads의 transpose에 의해 shape이 결정됨)
    query = self.split_heads(query, batch_size) # (batch_size, num_heads, seq_len, d_model/num_heads) 이것이 결과 shape
    key = self.split_heads(key, batch_size) # (batch_size, num_heads, seq_len, d_model/num_heads)
    value = self.split_heads(value, batch_size) # (batch_size, num_heads, seq_len, d_model/num_heads)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    outputs = self.dense(concat_attention)

    return outputs # 최종 결과 리턴

In [16]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      # 활성화 함수 relu는 첫번째 층에만 배치한다.
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])
